# ML Project 3  
Joseph Bentivegna  
Professor Keene  
10/24/17  

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# setup observations
numObs = 100
ratio = .5
N1 = int(numObs*ratio)
N2 = int(numObs*(1-ratio))

# setup mean and cov
C1Mean = np.array([1, 1])
C2Mean = np.array([-1, -1])
SD = np.identity(2)

C1class = np.ones(N2)
C2class = np.zeros(N1)

C1points = np.random.multivariate_normal(C1Mean, SD, N1)
C2points = np.random.multivariate_normal(C2Mean, SD, N2)

C1 = np.append(C1points.transpose(), np.atleast_2d(C1class), axis=0)
C2 = np.append(C2points.transpose(), np.atleast_2d(C2class), axis=0)

# setup combined matrix with truth values
combined = np.append(C1, C2, axis = 1)
obs = np.random.permutation(combined.T).T

## Gaussian Generative Model

In [3]:
# solve for mu and cov
mu1x = (np.sum(obs[0,:] * obs[2,:]))/N1
mu1y = (np.sum(obs[1,:] * obs[2,:]))/N2
mu1 = np.array([mu1x, mu1y])

mu2x = (np.sum(obs[0,:] * (1 - (obs[2,:]))))/N1
mu2y = (np.sum(obs[1,:] * (1 - (obs[2,:]))))/N2
mu2 = np.array([mu2x, mu2y])

hund1 = np.ones(N1)
hund2 = np.ones(N2)
mu1hund = np.array([hund1*mu1x,hund1*mu1y])
mu2hund = np.array([hund2*mu2x,hund2*mu2y])

omit0 = ((1 <= obs[2,:]))
omit1 = ((0 >= obs[2,:]))

S1 = ((obs[0:2, omit0] - mu1hund) @ (obs[0:2, omit0] - mu1hund).T)/N1
S2 = ((obs[0:2, omit1] - mu2hund) @ (obs[0:2, omit1] - mu2hund).T)/N2
S = (N1/numObs)*S1 + (N2/numObs)*S2

In [4]:
# solve for pi
pie1 = N1/(N1+N2)
pie2 = N2/(N1+N2)

# solve for alpha
mu1mu2 = np.array([mu1x-mu2x, mu1y-mu2y])
w = np.linalg.inv(S)@(mu1mu2).T
w0 = (-1/2)*(mu1)@np.linalg.inv(S)@(mu1).T + (1/2)*(mu2)@np.linalg.inv(S)@(mu2).T + np.log(pie1/pie2)
a = (w.T@obs[0:2,:]) + w0

# solve for sigmoid
sig = 1/(1+(np.exp(-a)))

# determine correctness
check = np.vstack([obs, sig])
count = 0

for i in range(0,numObs):
    if check[2,i] == 1 and check[3,i] > .5:
        count += 1
    if check[2,i] == 0 and check[3,i] < .5:
        count += 1 
    
perCor = count/numObs

print("Gaussian Generative Model % Correct: ", perCor)

Gaussian Generative Model % Correct:  0.95


## Logistic Regression Classifier

In [5]:
# setup design matrix based on linear basis function [1, X, Y]
phi = np.concatenate((np.atleast_2d(np.ones(numObs)).T, np.atleast_2d(obs[0,:]).T, np.atleast_2d(obs[1,:]).T), axis=1)
# setup prior input
wold = np.atleast_2d(np.concatenate((np.ones(1), np.ones(1), np.ones(1)))).T

#iterative step (20 iterations)
for i in range(0, 20):

    a = wold.T @ phi.T                  # alpha formula (w.T)(phi)
    y = 1/(1+(np.exp(-a)))              # y = sigmoid = 1/(1+exp(-a))
    R = np.zeros(numObs)                # R = revised weighting matrix
    for j in range(0,numObs):
        R[j] = y[0,j]*(1-y[0,j])
    R = np.diag(R)
    z = (phi@wold) - (np.linalg.inv(R) @ (y-obs[2,:]).T)
    wold = (np.linalg.inv(phi.T@R@phi))@phi.T@R@z

# solve for sigma with new final weights
af = wold.T @ phi.T
sigf = 1/(1+(np.exp(-a)))

# determine correctnes
check = np.vstack([obs, sigf])
count = 0

for i in range(0,numObs):
    if check[2,i] == 1 and check[3,i] > .5:
        count += 1
    if check[2,i] == 0 and check[3,i] < .5:
        count += 1

perCor = count/numObs

print("Logistic Regression Classifier % Correct: ", perCor)

Logistic Regression Classifier % Correct:  0.96
